In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import random

## Setting up selenium and testing on the first site

In [3]:
#Setting up the selenium driver
chromedriver = './chromedriver/'
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe")

In [4]:
#yahoo sites to retrieve comments from:
yahoo_sites =['https://sg.news.yahoo.com/yahoo-poll-good-thing-replace-streaming-subject-based-banding-230057280.html',
             'https://sg.news.yahoo.com/budget-2019-debate-subject-based-banding-replace-streaming-secondary-schools-2024-052556148.html',
             'https://sg.news.yahoo.com/yahoo-poll-issue-express-child-mixed-normal-stream-kids-230015842.html',
             'https://sg.news.yahoo.com/remove-streaming-secondary-schools-reduce-social-stratification-louis-ng-120612690.html']

In [5]:
#Go to website
driver.get(yahoo_sites[0])

In [6]:
#Retrieve Title
title = driver.find_element_by_css_selector("#SideTop-0-HeadComponentTitle > h1").text
title

'Yahoo Poll: Is it a good thing to replace streaming with subject-based banding?'

In [7]:
#Retrieve article text
article_text = driver.find_element_by_css_selector('.canvas-body').text

In [8]:
#Take a look at the text
article_text

'From 2024, secondary school students will no longer be streamed into Normal and Express classes, said Education Minister Ong Ye Kung on Tuesday (5 March). Instead, subject-Based Banding (SBB) will see students taking higher- and lower-level subjects based on their ability.\nAccording to Ong, the Full SBB will also enable schools to “reshape the social environment in schools to benefit their students”. Some 25 secondary schools will implement this starting in 2020.\nIs it a good thing for Singapore to replace secondary school streaming with subject-based banding? Have your say in our poll and leave a comment below.\n\nRelated stories:\nBudget 2019 debate: Subject-Based Banding to replace streaming in secondary schools by 2024\nRemove streaming in secondary schools to reduce social stratification: Louis Ng\n '

In [9]:
#Remove the footer section
article = article_text.split('\nRelated stor')[0]
article = article.split('\nOther Singapore stories')[0]

In [10]:
article

'From 2024, secondary school students will no longer be streamed into Normal and Express classes, said Education Minister Ong Ye Kung on Tuesday (5 March). Instead, subject-Based Banding (SBB) will see students taking higher- and lower-level subjects based on their ability.\nAccording to Ong, the Full SBB will also enable schools to “reshape the social environment in schools to benefit their students”. Some 25 secondary schools will implement this starting in 2020.\nIs it a good thing for Singapore to replace secondary school streaming with subject-based banding? Have your say in our poll and leave a comment below.\n'

In [11]:
#Reveal comments
comments_button = driver.find_element_by_css_selector('.comments-title')
comments_button.click()

In [12]:
#Click "Show More" till all comments are shown
while True:
    try:
        show_more = driver.find_element_by_css_selector("button.Ff\(ss\)")
        show_more.click()
    except:
        break

In [26]:
#Retrieve all comments
comments = []
i=1
while True:
    try:
        comments.append(driver.find_element_by_css_selector("li.comment:nth-child({}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)".format(i)).text)
        i+=1
    except:
        break

In [27]:
len(comments)

34

In [28]:
comments[0]

"What's the point of studying so much ended up working as a Cleaner and taxi driver for university graduates while the rest of the position fill up by foreign talents who work to apply for Singapore citizenship then went back to China to join People Liberation Army and dropped Singapore Citizenship"

In [29]:
comments[30]

'When we are stringent to ourselves but when it comes to final outcome - Singaporean lose out. Foreigners always stand a bigger chance. They are the winners. Why? They are cheaper, no need NS, no need older workers, no need student holiday job workers, no need reservists, no need to pay CPF, no expensive housing, no expensive education, no Kai Su ways to give the best to their children starting from nursery and so on.'

In [30]:
#Retrieve replies
ActionChains(driver).move_to_element(driver.find_element_by_css_selector('li.comment:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)')).perform()
sleep(3)
replies = []
i=2
try:
    replies_button = driver.find_element_by_css_selector("li.Mt\(5px\):nth-child({}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > button:nth-child(2)".format(i))
    replies_button.click()
    reply_num = 1
    sleep(1)
    while True:
        replies.append(driver.find_element_by_css_selector("li.comment:nth-child({}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(3) > ul:nth-child(2) > li:nth-child({}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)".format(i,reply_num)).text)
        reply_num+=1
except:
    pass

In [31]:
replies

['Seems to be that the purpose of this system is to allow students to learn at their own pace. Maybe better students get to graduate earlier? (Is there such a thing?)']

<div class ='alert alert-block alert-warning'>
    Replies don't show up unless the the page is scrolled back to the start of the comments.
</div>

In [13]:
#Create lists to append to
comments = []
is_reply = []
#Set counter for comments
i=1

#Scroll back to the top of the comments
ActionChains(driver).move_to_element(driver.find_element_by_css_selector('li.comment:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)')).perform()
sleep(3)

#Iterate through the comments
while True:
    try:
        #Append comment to the list
        comments.append(driver.find_element_by_css_selector("li.comment:nth-child({}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)".format(i)).text)
        is_reply.append(0)
        #Retrieve replies (if any)
        try:
            replies_button = driver.find_element_by_css_selector("li.Mt\(5px\):nth-child({}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > button:nth-child(2)".format(i))
            replies_button.click()
            sleep(2)
            #counter for replies
            reply_num = 1
            
            #Retrieve replies
            while True:
                comments.append(driver.find_element_by_css_selector("li.comment:nth-child({}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(3) > ul:nth-child(2) > li:nth-child({}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)".format(i,reply_num)).text)
                is_reply.append(1)
                reply_num+=1
        
        #increase comment counter and restart the loop 
        except:
            i+=1
            continue
        
    except:
        break

In [14]:
test_df = pd.DataFrame({'comment':comments,'is_reply':is_reply})

In [15]:
test_df

,comment,is_reply
0,What's the point of studying so much ended up ...,0
1,No matter what kind of streaming or subject ba...,0
2,Seems to be that the purpose of this system is...,1
3,This feels like just another diversion from RE...,0
4,Isn’t a “real” issue the boxing of kids into s...,1
5,Streaming was started by that tancho tan Tony ...,0
6,"Will they admit it, that it is a failure? Perh...",1
7,the issue is not how the cake is being cut. th...,0
8,Who started the secondary school streaming in ...,0
9,@RneGan Who else? Our Great Founding Leeder!,1


## Retrieve the comments and replies from all the pages

In [16]:
#Create lists and DataFrames for appending to
titles = []
text = []
comments_df = pd.DataFrame({'article_id':[],'comments':[],'is_reply':[]})

#Run through the articles and retrieve the various information
for i in range(len(yahoo_sites)):
    
    #Go to site
    driver.get(yahoo_sites[i])
    
    #Retrieve Title
    title = driver.find_element_by_css_selector("#SideTop-0-HeadComponentTitle > h1").text
    title
    #Add title to the list
    titles.append(title)
    
    #Retrieve article text
    article_text = driver.find_element_by_css_selector('.canvas-body').text
    
    #Remove the footer section
    article_text = article_text.split('\nRelated stor')[0]
    article_text = article_text.split('\nOther Singapore stories')[0]
    #Add text to the list
    text.append(article_text)
    sleep(random.randint(5,10))
    
    #Reveal comments
    comments_button = driver.find_element_by_css_selector('.comments-title')
    comments_button.click()
    sleep(3)
    
    #Click "Show More" till all comments are shown
    while True:
        try:
            show_more = driver.find_element_by_css_selector("button.Ff\(ss\)")
            show_more.click()
        except:
            break
    sleep(random.randint(4,11))
    
    #Retrieve all comments
    comments = []
    comment_num = 1
    is_reply = []
    
    #Scroll back to first comment
    ActionChains(driver).move_to_element(driver.find_element_by_css_selector('li.comment:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)')).perform()
    
    #Iterate through the comments
    while True:
        try:
            #Retrieve the comment
            comments.append(driver.find_element_by_css_selector("li.comment:nth-child({}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)".format(comment_num)).text)
            is_reply.append(0)
            
            #Expand replies (if any)
            try:
                replies_button = driver.find_element_by_css_selector("li.Mt\(5px\):nth-child({}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > button:nth-child(2)".format(comment_num))
                replies_button.click()
                sleep(2)

                #counter for replies
                reply_num = 1

                #Retrieve replies
                while True:
                    comments.append(driver.find_element_by_css_selector("li.comment:nth-child({}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(3) > ul:nth-child(2) > li:nth-child({}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)".format(comment_num,reply_num)).text)
                    is_reply.append(1)
                    reply_num+=1
            #
            increase comment counter and restart the loop
            except:
                comment_num+=1
                continue

        except:
            break
    
    #convert all comments into a DataFrame
    com_df = pd.DataFrame({'article_id':[i]*len(comments),'comments':comments,'is_reply':is_reply})
    #Concatenate the new comments to existing ones
    comments_df = pd.concat([comments_df,com_df],axis=0)
    sleep(random.randint(2,6))

#Create a DataFrame of the article ids, title and text.
articles_df = pd.DataFrame({'article_id':range(len(yahoo_sites)),'title':titles,'text':text})

#Save the DataFrames of article information and comments
articles_df.to_csv('./articles.csv')
comments_df.to_csv('./comments.csv')

In [17]:
#Check the articles
articles_df

,article_id,title,text
0,0,Yahoo Poll: Is it a good thing to replace stre...,"From 2024, secondary school students will no l..."
1,1,Budget 2019 debate: Subject-Based Banding to r...,"From 2024, secondary school students will no l..."
2,2,Yahoo Poll: Would it be an issue for you if yo...,A Singapore mother of a three has suddenly com...
3,3,Remove streaming in secondary schools to reduc...,The Ministry of Education (MOE) should conside...


In [25]:
#Check the comments
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 68
Data columns (total 3 columns):
article_id    265 non-null float64
comments      265 non-null object
is_reply      265 non-null float64
dtypes: float64(2), object(1)
memory usage: 8.3+ KB


In [24]:
comments_df['is_reply'].value_counts()

0.0    184
1.0     81
Name: is_reply, dtype: int64

In [26]:
comments_df.head()

,article_id,comments,is_reply
0,0.0,What's the point of studying so much ended up ...,0.0
1,0.0,No matter what kind of streaming or subject ba...,0.0
2,0.0,Seems to be that the purpose of this system is...,1.0
3,0.0,This feels like just another diversion from RE...,0.0
4,0.0,Isn’t a “real” issue the boxing of kids into s...,1.0


In [27]:
comments_df.tail()

,article_id,comments,is_reply
64,3.0,Good. another own self check own self. LoL,0.0
65,3.0,Louis is 100% correct! I support his thinking ...,0.0
66,3.0,"Election tactics lah, now act like good guy so...",1.0
67,3.0,One of the many useless jiao liao be in the house,0.0
68,3.0,Louis Ng is obviously an IDIOT.,0.0
